# Test LLM use with Gigachat

In [67]:
# !pip install gigachat
# !pip install gigachain
# !pip install pypdf2
# !pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00a 0:00:01


In [68]:
CREDENTIALS = 'Yjg4MTQzMmUtNDAwMS00NDk0LThjOGUtNmU5ZWQ2YzQ4NDQ2OmQ4MWMxZGZiLTFmNGYtNDk5NS05OGQzLTBiMzYyYWJmNjk3OA=='

## 1. Test that Gigachat works

In [69]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

# Авторизация в сервисе GigaChat
chat = GigaChat(credentials=CREDENTIALS, 
                verify_ssl_certs=False,
                scope='GIGACHAT_API_CORP',
                model="GigaChat-Pro",
                )

# messages = [
#     SystemMessage(
#         content="Ты эмпатичный бот-психолог, который помогает пользователю решить его проблемы."
#     )
# ]
# i = 0
# while i < 1:
#     # Ввод пользователя
#     user_input = input("User: ")
#     messages.append(HumanMessage(content=user_input))
#     res = chat(messages)
#     messages.append(res)
#     # Ответ модели
#     print("Bot: ", res.content)
#     i+=1

## Recreate PDF retrieval with prompt and some test pdf

In [75]:
from PyPDF2 import PdfReader

reader = PdfReader("../data/papers/10.1002@solr.201900061.pdf")
# pages = loader.load_and_split()

raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text
        

In [77]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GigaChatEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter
)

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

texts = text_splitter.split_text(raw_text)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200,
# )
# documents = text_splitter.split_documents(pages)
embeddings = GigaChatEmbeddings(credentials=CREDENTIALS, verify_ssl_certs=False, scope='GIGACHAT_API_CORP')

In [80]:
faiss_index = FAISS.from_texts(texts=texts, embedding=embeddings)

In [81]:
from langchain.chains.question_answering import load_qa_chain

In [88]:
chain = load_qa_chain(chat, chain_type="stuff")

In [90]:
query = "Please find HTM layers that were used in this paper. List them in the list"
docs = faiss_index.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Spiro HTM, Spiro-CB, Spiro-THF'

In [91]:
query = "For each of these {Spiro HTM, Spiro-CB, Spiro-THF} report efficiency (PCE or optimized efficiency or η). Put that data in the markdown table with columns 'HTM' - 'PCE'"
docs = faiss_index.similarity_search(query)
chain.run(input_documents=docs, question=query)

'| HTM        | PCE       |\n|-------------|-----------|\n| Spiro HTM   | N/A       |\n| Spiro-CB    | 15.27%    |\n| Spiro-THF   | 16.94%    |'

In [92]:
print('| HTM        | PCE       |\n|-------------|-----------|\n| Spiro HTM   | N/A       |\n| Spiro-CB    | 15.27%    |\n| Spiro-THF   | 16.94%    |')

| HTM        | PCE       |
|-------------|-----------|
| Spiro HTM   | N/A       |
| Spiro-CB    | 15.27%    |
| Spiro-THF   | 16.94%    |
